# Merge all auctions into one

Combines several datasets and do some simple analyses.  


### User variables

In [1]:
# dates of auction
# format: yyyy-mm; yyyy: year, mm: month (%Y-%m)
AuctionDates = ['2017-03','2017-04','2017-05','2017-06','2017-07','2017-08','2017-09','2017-10','2017-11','2017-12',
                '2018-01','2018-02','2018-03','2018-04','2018-05','2018-06','2018-07','2018-08','2018-09','2018-10','2018-11','2018-12',
                '2019-01','2019-02','2019-03','2019-04','2019-05','2019-06']

### Import modules 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import re
plt.style.use('ggplot')

### Load all data

Auctions are saved monthly in a .pkl file

In [3]:
print('load data')
data = dict()
for AuctionDate in AuctionDates:
    # read data
    fn = '../data/rdw-data-{:s}.pkl'.format(
        AuctionDate
    )
    print(fn)
    df = pd.read_pickle(fn)
    data[AuctionDate] = df

print('\nlast rows of last file')
data[AuctionDates[-1]].tail()

load data
../data/rdw-data-2017-03.pkl
../data/rdw-data-2017-04.pkl
../data/rdw-data-2017-05.pkl
../data/rdw-data-2017-06.pkl
../data/rdw-data-2017-07.pkl
../data/rdw-data-2017-08.pkl
../data/rdw-data-2017-09.pkl
../data/rdw-data-2017-10.pkl
../data/rdw-data-2017-11.pkl
../data/rdw-data-2017-12.pkl
../data/rdw-data-2018-01.pkl
../data/rdw-data-2018-02.pkl
../data/rdw-data-2018-03.pkl
../data/rdw-data-2018-04.pkl
../data/rdw-data-2018-05.pkl
../data/rdw-data-2018-06.pkl
../data/rdw-data-2018-07.pkl
../data/rdw-data-2018-08.pkl
../data/rdw-data-2018-09.pkl
../data/rdw-data-2018-10.pkl
../data/rdw-data-2018-11.pkl
../data/rdw-data-2018-12.pkl
../data/rdw-data-2019-01.pkl
../data/rdw-data-2019-02.pkl
../data/rdw-data-2019-03.pkl
../data/rdw-data-2019-04.pkl
../data/rdw-data-2019-05.pkl
../data/rdw-data-2019-06.pkl

last rows of last file


,Source,Title,Price,Draw,Raw_text,N_images,Images,Note,LotNr,LotType,...,rdw_uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_3,rdw_uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_4,rdw_uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_5,rdw_uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_6,rdw_uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_7,rdw_uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_19,rdw_versnellingsbak_uitvoering_aantal_versnellingen_bovengrens_1,rdw_versnellingsbak_uitvoering_aantal_versnellingen_ondergrens_1,rdw_versnellingsbak_uitvoering_type_versnellingsbak_1,rdw_TimeStamp_y
2019-6-9155,http://www.domeinenrz.nl/catalogi/verkoop_bij_...,Kavel K1900069155,644.0,False,"[Type 7df-ajt1m, Kenteken 94-VZ-GK, Km-stand o...",3,[http://www.domeinenrz.nl/ufc/static/155896521...,False,K1900069155,Bedrijfswagen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-6-9157,http://www.domeinenrz.nl/catalogi/verkoop_bij_...,Kavel K1900069157,1025.0,False,"[Type dv75, Datum eerste toelating 26.01.2016,...",2,[http://www.domeinenrz.nl/ufc/static/155896521...,False,K1900069157,Aanhangwagen (Enkelasser),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-6-9158,http://www.domeinenrz.nl/catalogi/verkoop_bij_...,Kavel K1900069158,2431.0,False,"[Type 116i, Kenteken 18-TD-JG, Afgelezen km-st...",3,[http://www.domeinenrz.nl/ufc/static/155896521...,False,K1900069158,Personenauto,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190611
2019-6-9159,http://www.domeinenrz.nl/catalogi/verkoop_bij_...,Kavel K1900069159,2269.0,False,"[Bouwjaar 2005, Draaiurenstand onbekend, Inhou...",4,[http://www.domeinenrz.nl/ufc/static/155896521...,False,K1900069159,CONTAINER (Merk onbekend),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-6-9700,http://www.domeinenrz.nl/catalogi/verkoop_bij_...,Kavel K1900069700,20015.0,False,"[Loopplanken en platforms, Ca. 2 stuks van 6 m...",9,[http://www.domeinenrz.nl/ufc/static/155896521...,False,K1900069700,STEIGERMATERIAAL EN TOEBEHOREN (Diverse afmeti...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Consolidate format
Older files are slightly different such as naming conventions etc. Here all results are consolidated into one format.

In [4]:
# Images

# print first lot
print('Fields with images look like this:\n')
print('{:7s} | {:7s} | {:14s} | {:7s}\n{:7s}-+-{:s}-+-{:s}-+-{:s}'.format('Month','Column','Type','1st row','-'*7,'-'*7,'-'*14,'-'*46))
for k in data:
    col_idx = pd.np.where([c.lower() in ['image','images'] for c in data[k].columns])[0]
    example = data[k].iloc[0,col_idx].values[0]
    print('{:7s} | {:7s} | {}'.format(k,data[k].columns[col_idx][0],type(example)),end='')
    if type(example) == str:
        print('  | "{} .. {}"'.format(example[0:20],example[-20:]))
    else:
        print(' | length {} '.format(len(example)))

Fields with images look like this:

Month   | Column  | Type           | 1st row
--------+---------+----------------+-----------------------------------------------
2017-03 | Image   | <class 'str'>  | "['http://www.domeine .. 1024/768/image.jpg']"
2017-04 | Image   | <class 'str'>  | "['http://www.domeine .. 1024/768/image.jpg']"
2017-05 | Image   | <class 'str'>  | "['http://www.domeine .. 1024/768/image.jpg']"
2017-06 | Image   | <class 'str'>  | "['http://www.domeine .. 1024/768/image.jpg']"
2017-07 | Image   | <class 'str'>  | "['http://www.domeine .. 1024/768/image.jpg']"
2017-08 | Image   | <class 'str'>  | "['http://www.domeine .. 1024/768/image.jpg']"
2017-09 | Image   | <class 'str'>  | "['http://www.domeine .. 1024/768/image.jpg']"
2017-10 | Image   | <class 'str'>  | "['http://www.domeine .. 1024/768/image.jpg']"
2017-11 | Images  | <class 'str'>  | "[http://www.domeinen .. /1024/768/image.jpg]"
2017-12 | Images  | <class 'str'>  | "[http://www.domeinen .. /1024/768/image.j

In [5]:
# convert string representation of list to real list
# "['image1', 'image2']"
for k in ['2017-03',
          '2017-04',
          '2017-05',
          '2017-06',
          '2017-07',
          '2017-08',
          '2017-09',
          '2017-10']:
    data[k]['Images'] = data[k]['Image'].apply(eval)
    data[k] = data[k].drop(columns=['Image'])

# convert string representation of list without quotes to real list
# "[image1, image2]"
for k in ['2017-11',
          '2017-12',
          '2018-01',
          '2018-02',
          '2018-03',
          '2018-04']:
    data[k]['Images'] = data[k]['Images'].apply(lambda s:eval(re.sub(r"^\[(.*)\]$",r"['\1']",re.sub(' *, *',"', '",s))))
    

In [6]:
# Other column name that changed in november 2017
for k in ['2017-03',
          '2017-04',
          '2017-05',
          '2017-06',
          '2017-07',
          '2017-08',
          '2017-09',
          '2017-10']:
    data[k] = data[k].rename(columns={
        'draw':'Draw',
        'nr':'LotNr',
        'raw':'Raw_text',
        'misc':'SupInfo',
        'footnote':'Note',
        'jfq':'jfc',
    })

# Merge all auctions into one data frame

In [7]:
# append data into one data frame
for i,k in enumerate(sorted(data.keys(),reverse=False)):
    if i == 0:
        df = data[k]
    else:
        oldcols = pd.np.setdiff1d(df.columns,data[k].columns)
        newcols = pd.np.setdiff1d(data[k].columns,df.columns)
        if oldcols.size > 0:
            print('fields not present in {:s}'.format(k))
            [print('\t{:s}'.format(c)) for c in oldcols]
        if newcols.size > 0:
            print('fields new in {:s}'.format(k))
            [print('\t{:s}'.format(c)) for c in newcols]
        
        df=df.append(data[k],sort=False).copy()
    print(k,df.shape)
            

2017-03 (234, 144)
fields not present in 2017-04
	rdw_oplegger_geremd
fields new in 2017-04
	rdw_uitstoot_deeltjes_zwaar
2017-04 (434, 145)
fields not present in 2017-05
	rdw_oplegger_geremd
	rdw_uitstoot_deeltjes_zwaar
2017-05 (561, 145)
fields not present in 2017-06
	rdw_oplegger_geremd
	rdw_uitstoot_deeltjes_zwaar
fields new in 2017-06
	rdw_nominaal_continu_maximumvermogen
2017-06 (804, 146)
2017-07 (993, 146)
fields not present in 2017-08
	rdw_nominaal_continu_maximumvermogen
	rdw_oplegger_geremd
	rdw_vervaldatum_tachograaf
	rdw_weggedrag_code
2017-08 (1238, 146)
fields not present in 2017-09
	rdw_nominaal_continu_maximumvermogen
	rdw_oplegger_geremd
	rdw_uitstoot_deeltjes_zwaar
2017-09 (1447, 146)
fields not present in 2017-10
	rdw_nominaal_continu_maximumvermogen
	rdw_oplegger_geremd
2017-10 (1626, 146)
fields not present in 2017-11
	disclaim3
	disclaim4
	hybrid
	import
	jfc
	no_igk
	no_nlreg193
	rdw_TimeStamp
	rdw_oplegger_geremd
	taxi
fields new in 2017-11
	N_images
	Source
	ea

# Continue with cars only

In [8]:
# select data
car=df[df.LotType == 'Personenauto'].copy()

In [9]:
# print lots that are cars per auction
prev = ['1999','0','0']
print('Car lots\nYear | Month | First ... Last index')
for ix in car.index:
    y,m,k = ix.split('-')
    if (y != prev[0]) or (m != prev[1]):
        if prev[0] == '1999':
            print('-----+-------+---------------------',end='')
        else:
            print('.. {:s}'.format(prev[2]),end='')
        print('\n{:4s} | {:>5s} |'.format(y,m),end='')
        prev[0] = y
        prev[1] = m
        print(' {:>5s} .'.format(k),end='')
    prev[2] = k
print('.. {:s}'.format(prev[2]),end='')
car.tail()

Car lots
Year | Month | First ... Last index
-----+-------+---------------------
2017 |     3 |  2000 ... 8318
2017 |     4 |  2000 ... 7309
2017 |     5 |  2200 ... 8025
2017 |     6 |  2000 ... 7311
2017 |     7 |  2200 ... 8178
2017 |     8 |  2001 ... 7304
2017 |     9 |  2200 ... 8187
2017 |    10 |  2000 ... 7171
2017 |    11 |  2200 ... 8305
2017 |    12 |  2000 ... 7308
2018 |     1 |  2200 ... 8170
2018 |     2 |  2000 ... 7184
2018 |     3 |  2200 ... 8338
2018 |     4 |  2000 ... 7336
2018 |     5 |  2200 ... 8226
2018 |     6 |  2000 ... 7315
2018 |     7 |  2200 ... 8311
2018 |     8 |  2000 ... 7328
2018 |     9 |  2200 ... 8183
2018 |    10 |  2000 ... 7319
2018 |    11 |  2200 ... 8326
2018 |    12 |  2000 ... 7361
2019 |     1 |  2201 ... 8329
2019 |     2 |  2001 ... 7264
2019 |     3 |  2200 ... 8318
2019 |     4 |  2000 ... 7314
2019 |     5 |  2200 ... 8352
2019 |     6 |  2201 ... 9158

,LotNr,Price,Title,LotCat,LotType,ItemBrand,ItemType,Mfdate,Mfyear,APKdate,...,rdw_carrosserie_carrosserietype_1,rdw_carrosserie_type_carrosserie_europese_omschrijving_1,rdw_carrosserie_specifiek_carrosserie_voertuig_nummer_code_volgnummer_1,rdw_carrosserie_specifiek_carrosserie_voertuig_nummer_europese_omschrijving_1,rdw_carrosserie_specifiek_carrosseriecode_1,d_lic,btw21,rdw_brandstof_co2_uitstoot_gewogen_1,rdw_brandstof_co2_uitstoot_gewogen_2,legguard
2019-6-9146,K1900069146,4509.0,Kavel K1900069146,Voertuigen en onderdelen,Personenauto,VOLKSWAGEN,polo,23.05.2012,,,...,AB,Hatchback,NaN,NaN,NaN,False,False,NaN,NaN,False
2019-6-9148,K1900069148,1234.0,Kavel K1900069148,Voertuigen en onderdelen,Personenauto,VOLKSWAGEN,transporter bestel 1.0 tdi 65,18.05.2001,,31.07.2019,...,BB,Bestelwagen,NaN,NaN,NaN,False,False,NaN,NaN,False
2019-6-9149,K1900069149,5555.0,Kavel K1900069149,Voertuigen en onderdelen,Personenauto,VOLKSWAGEN,golf,05.07.2013,,11.10.2019,...,AC,Stationwagen,NaN,NaN,NaN,False,False,NaN,NaN,False
2019-6-9152,K1900069152,2269.0,Kavel K1900069152,Voertuigen en onderdelen,Personenauto,OPEL,corsa,26.10.2006,,12.12.2019,...,AB,Hatchback,NaN,NaN,NaN,False,False,NaN,NaN,False
2019-6-9158,K1900069158,2431.0,Kavel K1900069158,Voertuigen en onderdelen,Personenauto,BMW,116i,15.07.2006,,,...,AC,Stationwagen,NaN,NaN,NaN,False,False,NaN,NaN,False


# Save file

In [10]:
# save data
fn = '../data/cars-from-all-auctions.pkl'
print(fn)
car.to_pickle(fn)

../data/cars-from-all-auctions.pkl


# Write example results to file
based on: https://stackoverflow.com/a/33869154

In [11]:
def pandas_df_to_markdown_table(df):
    fmt = ['-----' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    return df_formatted.to_csv(sep="|", index=False)


In [12]:
fn = '../assets/example-table-of-cars.md'
example = car.tail(10).copy()
# trim some long fields
example.rdwinfo = '.. rdw info ..'
example.Raw_text = '.. raw text ..'
example.SupInfo = '.. suplm. info. ..'
table_text = pandas_df_to_markdown_table(example)
with open(fn,'w') as file:
    file.write(table_text)
    
print('A markdown table is available as\n\t{}'.format(fn))

A markdown table is available as
	../assets/example-table-of-cars.md
